<a href="https://colab.research.google.com/github/lainey-reed/Textbook/blob/master/Colab/AC_Textbook_Chapter7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 7: Rapid Mix

All of the code examples in this colab file are from [Chapter 7: Rapid Mix](https://aguaclara.github.io/Textbook/Rapid_Mix/RM_Intro.html)

In [ ]:
!pip install aguaclara

## Example 1: Maximum Velocity Gradients

The example below is from the section [Conventional Mechanical Rapid Mix: Maximum Velocity Gradients](https://aguaclara.github.io/Textbook/Rapid_Mix/RM_Intro.html#maximum-velocity-gradients)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker
from aguaclara.core.units import unit_registry as u
import aguaclara as ac


Mix_HRT = np.array([0.5,15,25,35,85])*u.s
Mix_G = np.array([4000,1500,950,850,750])/u.s
Mix_CP = np.multiply(Mix_HRT, np.sqrt(Mix_G))
Mix_Gt = np.multiply(Mix_HRT, Mix_G)
# added line to create temperature variable
temperature = 22 * u.degC
Mix_EDR = (Mix_G**2*ac.core.physchem.viscosity_kinematic(temperature))

fig, ax = plt.subplots()
ax.plot(Mix_G.to(1/u.s),Mix_HRT.to(u.s),'o')
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.f'))
ax.xaxis.set_major_formatter(ticker.FormatStrFormatter('%.f'))
ax.set(xlabel='Velocity gradient (Hz)', ylabel='Residence time (s)')

#fig.savefig(imagepath+'Mechanical_RM_Gt')
plt.show()

## Examples: Rapid Mix Design

The next two code examples are from the [Rapid Mix Deisign](https://aguaclara.github.io/Textbook/Rapid_Mix/RM_Design.html) section.

In [ ]:
n_points = 50
h_e_graph = np.logspace(-1,1,n_points) * u.m

v_channel = 0.45 * u.m/u.s
def Q_per_mixer(t_eddy):
  return (u.gravity * h_e_graph * t_eddy**2 * v_channel).to(u.L/u.s)

plt.plot(h_e_graph,Q_per_mixer(1*u.s),linewidth=3)
plt.plot(h_e_graph,Q_per_mixer(0.3*u.s),linewidth=3)
plt.plot(h_e_graph,Q_per_mixer(0.1*u.s),linewidth=3)
plt.xscale("log")
plt.yscale("log")
plt.xlabel('Head loss (m)')
plt.ylabel('Flow rate (L/s)')
plt.grid(which='both')
plt.legend(['1 s','0.3 s','0.1 s'])
plt.gca().yaxis.set_major_formatter(ticker.ScalarFormatter())
plt.gca().xaxis.set_major_formatter(ticker.ScalarFormatter())

In [ ]:
Head_loss_max = 20 * u.cm
t_eddy = 0.3 * u.s
L_eddy = (t_eddy * np.sqrt(u.gravity * Head_loss_max)).to(u.m)
print('The spacing between injection ports is',L_eddy)


#expanded velocity
v_channel = 0.45 * u.m/u.s
Q_per_mixer = (v_channel * L_eddy**2).to(u.L/u.s)
print('The flow rate of raw water per chemical injection point is',Q_per_mixer)

Q=20000/7 * u.L/u.s #Fairmont design
A_channel = Q/v_channel
n_ports = (A_channel/L_eddy**2).to(u.dimensionless)
print('The number of injection ports is',(np.round(n_ports)).magnitude)

Pi_A = np.sqrt(2*u.gravity*Head_loss_max)/v_channel + 1
print('The expansion ratio is',Pi_A)
v_jet = v_channel * Pi_A
w_contraction = L_eddy/Pi_A
print('The width of the contractions is',w_contraction)

D_cyl = L_eddy - w_contraction
print('The diameter of the cylinder is',D_cyl)

## Rapid Mix Appendix C: Examples

The following cells of code are from [Rapid Mix Appendix C: Examples](https://aguaclara.github.io/Textbook/Rapid_Mix/RM_Examples.html)

### Example: pH Adjustment

This code is from [Example: pH Adjustment](https://aguaclara.github.io/Textbook/Rapid_Mix/RM_Examples.html#example-ph-adjustment)**bold text**

In [ ]:
from aguaclara.core.units import unit_registry as u
import aguaclara.research.environmental_processes_analysis as epa
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

"""define molecular weights"""
m_Ca = 40.078*u.g/u.mol
m_C = 12.011*u.g/u.mol
m_O = 15.999*u.g/u.mol
m_Na = 22.99*u.g/u.mol
m_H = 1.008*u.g/u.mol
m_CaCO3 = m_Ca+m_C+3*m_O
m_Na2CO3 = 2*m_Na+m_C+3*m_O
m_NaHCO3 = m_Na+m_H+m_C+3*m_O
m_NaOH = m_Na+m_O+m_H

"""Raw water characteristics"""
pH_0 = 5.91
ANC_0 = (24.5 * u.mg/u.L/m_CaCO3).to(u.mmol/u.L)
ANC_0

def total_carbonates_closed(pH, ANC):
    """This function calculates total carbonates for a closed system given pH and ANC

    Parameters
    ----------
    pH : float
        pH of the sample
    ANC: float
        acid neutralizing capacity of the sample
    Returns
    -------
    The total carbonates of the sample
    Examples
    --------
    >>> total_carbonates_closed(1*u.mmol/u.L,8)
    1.017 mole/liter
    """
    return (ANC - epa.Kw/epa.invpH(pH) + epa.invpH(pH)) / (epa.alpha1_carbonate(pH) + 2 * epa.alpha2_carbonate(pH))


CT_0 = total_carbonates_closed(pH_0,ANC_0)


""" calculate the amount of base that must be added to reach a target pH"""

def pH_adjust(pH_0,ANC_0,Pi_base,Pi_CO3,Pi_Al,C_Al,pH_target):
  """This function calculates the required base (or acid) to adjust the pH to a target value. The buffering capacity is assumed to be completely due to carbonate species. The initial carbonate concentration is calculated based on the initial pH and the initial ANC.

  Parameters
  ----------
  pH_0: float
      pH of the sample
  ANC_0: float
      acid neutralizing capacity (Alkalinity) of the sample in eq/L.
  Pi_base: float
    equivalents of ANC per mole of base (or acid)
  Pi_CO3: float
    mole of carbonate per mole of base (or acid)
  Pi_Al : float
    equivalents of ANC per mole of aluminum coagulant
  C_Al
    concentration of aluminum coagulant in moles/L
  pH_target: float
    pH goal
  Returns
  -------
  The required concentration of base (or acid) in millimoles/L
  Examples
  --------
  >>> pH_adjust(5.91,0.2*u.mmol/u.L,1,1,0,0,7)
  2.2892822041250924 millimole/liter
  >>> pH_adjust(7,0.2*u.mmol/u.L,1,1,0,0,0,0,7)
  0.0 millimole/liter
  >>> pH_adjust(7,0*u.mmol/u.L,1,0,-3,1*u.mmol/u.L,7)
  3.0 millimole/liter
  """
  CT_0 = total_carbonates_closed(pH_0,ANC_0)
  B_num = CT_0 * (epa.alpha1_carbonate(pH_target) + 2 * epa.alpha2_carbonate(pH_target)) + epa.Kw/epa.invpH(pH_target) - epa.invpH(pH_target) - ANC_0 - Pi_Al*C_Al
  B_den = Pi_base - Pi_CO3*(epa.alpha1_carbonate(pH_target) + 2 * epa.alpha2_carbonate(pH_target))
  return (B_num/B_den).to(u.mmol/u.L)


"""target pH"""
pH_target = 7

Pi_base_Na2CO3 = 2
Pi_CO3_Na2CO3 = 1

Pi_base_NaHCO3 = 1
Pi_CO3_NaHCO3 = 1

Pi_base_NaOH = 1
Pi_CO3_NaOH = 0

C_Na2CO3 = pH_adjust(pH_0,ANC_0,Pi_base_Na2CO3,Pi_CO3_Na2CO3,0,0,pH_target)

C_NaHCO3 = pH_adjust(pH_0,ANC_0,Pi_base_NaHCO3,Pi_CO3_NaHCO3,0,0,pH_target)
C_NaOH = pH_adjust(pH_0,ANC_0,Pi_base_NaOH,Pi_CO3_NaOH,0,0,pH_target)

"""Display results in a pandas table"""
base = ["NaOH","NaHCO3","Na2CO3"]
myindex = ["[mmoles/L]","[mg/L]"]
row1 = [C_Na2CO3.magnitude,C_NaHCO3.magnitude,C_NaOH.magnitude]
row2 = [(C_Na2CO3*m_Na2CO3).to(u.mg/u.L).magnitude,(C_NaHCO3*m_NaHCO3).to(u.mg/u.L).magnitude,(C_NaOH*m_NaOH).to(u.mg/u.L).magnitude]
df = pd.DataFrame([row1,row2],index=myindex,columns=base)
print(df.round(2))

"""Graph the base concentration required as a function of the target pH"""
pH_graph = np.linspace(6,7,50)
C_Na2CO3 = pH_adjust(pH_0,ANC_0,Pi_base_Na2CO3,Pi_CO3_Na2CO3,0,0,pH_graph)
C_NaHCO3 = pH_adjust(pH_0,ANC_0,Pi_base_NaHCO3,Pi_CO3_NaHCO3,0,0,pH_graph)
C_NaOH = pH_adjust(pH_0,ANC_0,Pi_base_NaOH,Pi_CO3_NaOH,0,0,pH_graph)

fig, ax = plt.subplots()

ax.plot(pH_graph,C_NaHCO3)
ax.plot(pH_graph,C_Na2CO3)
ax.plot(pH_graph,C_NaOH)
#imagepath = 'Rapid_Mix/Images/'
ax.set(xlabel='pH target', ylabel='Base concentration (mmole/L)')
ax.legend(["sodium bicarbonate","sodium carbonate","sodium hydroxide"])
#fig.savefig(imagepath+'mole_base_for_target_pH')
plt.show()

fig, ax = plt.subplots()
ax.plot(pH_graph,(C_Na2CO3*m_Na2CO3).to(u.mg/u.L))
ax.plot(pH_graph,(C_NaOH*m_NaOH).to(u.mg/u.L))
ax.set(xlabel='pH target', ylabel='Base concentration (mg/L)')
ax.legend(["sodium carbonate","sodium hydroxide"])
#fig.savefig(imagepath+'mg_base_for_target_pH')
plt.show()

### Example: LFOM and Coagulant Injection

The following example can be found [here.](https://aguaclara.github.io/Textbook/Rapid_Mix/RM_Examples.html#lfom-and-coagulant-injection-sizing)

In [ ]:
from aguaclara.core.units import unit_registry as u
import aguaclara.research.environmental_processes_analysis as epa
import numpy as np
import matplotlib.pyplot as plt
import aguaclara.research.floc_model as fm
from aguaclara.design.lfom import LFOM
from aguaclara.core import pipes

#from matplotlib.ticker import FormatStrFormatter
#imagepath = 'AguaClara Water Treatment Plant Design/Rapid Mix/Images/'

Q_plant = 120 * u.L / u.s
HL_LFOM = 20 * u.cm
Pi_LFOM_safety = 1.2
SDR_LFOM = 26

ND_LFOM = LFOM(q=Q_plant,hl=HL_LFOM).pipe_nd
print(ND_LFOM, '(',ND_LFOM.to(u.cm), ')')

L_flow = pipes.ID_SDR(ND_LFOM,SDR_LFOM)
L_flow

In [ ]:
LFOM(q=20 * u.L / u.s, hl=20 * u.cm).pipe_nd

### Example: Energy dissapation in a straight pipe

The following examples can be found [here.](https://aguaclara.github.io/Textbook/Rapid_Mix/RM_Examples.html#example-problem-energy-dissipation-rate-in-a-straight-pipe)

In [ ]:
import aguaclara.core.pipes as pipes
import aguaclara.core.physchem as pc
from aguaclara.core.units import unit_registry as u
import aguaclara.core.materials as mat
T_water=0*u.degC
Pipe_roughness = mat.PVC_PIPE_ROUGH
Pipe_roughness
Nu_water = pc.viscosity_kinematic(T_water)
Q_pipe = 120 * u.L/u.s
ND_pipe = 24*u.inch
SDR_pipe = 26
ID_pipe = pipes.Pipe(ND_pipe,SDR_pipe).id_sdr
f_pipe = pc.fric(Q_pipe,ID_pipe,Nu_water,Pipe_roughness)
N_pipe_diameters = (2/f_pipe)**(1/3)
N_pipe_diameters
"""The minimum length for mixing is thus"""
L_mixing = ID_pipe*N_pipe_diameters
print('The minimum distance required for mixing across the diameter of the pipe is ',L_mixing.to_base_units())
v_lfom = (Q_plant/pc.area_circle(pipes.Pipe(ND_LFOM,SDR_LFOM).id_sdr)).to_base_units()
print(v_lfom)
t_mixing = (L_mixing/v_lfom).to(u.s)
t_mixing

In [ ]:
C_PACl_stock = 70 *u.g/u.L
C_PACl_dose_max = 10 * u.mg/u.L
Q_PACl_max = (Q_plant*C_PACl_dose_max/C_PACl_stock).to(u.mL/u.s)
print(Q_PACl_max)

In [ ]:
HL_Coag_injection = 2 * u.cm
v_Coag_injection = ((2 * u.gravity * HL_Coag_injection)**0.5).to(u.m/u.s)
print(v_Coag_injection)
D_Coag_injection_min = pc.diam_circle(Q_PACl_max/v_Coag_injection)
print(D_Coag_injection_min.to(u.mm))

## Rapid Mix Theory and Future Work

The following examples are from the section [Rapid Mix Theory and Future Work](https://aguaclara.github.io/Textbook/Rapid_Mix/RM_Theory_and_Future_Work.html)

### Example: Diffusion Band Thickness vs Velocity Gradient

This code can be found at the subsection [Diffustion band thickness](https://aguaclara.github.io/Textbook/Rapid_Mix/RM_Theory_and_Future_Work.html#diffusion-band-thickness)

In [ ]:
import aguaclara.core.physchem as pc
from aguaclara.core.units import unit_registry as u
import aguaclara.research.floc_model as fm
import numpy as np
import matplotlib.pyplot as plt

"""I needed to attach units to material properties due to a bug in floc_model. This will need to be fixed when floc_model is updated."""
#imagepath = 'Rapid_Mix/Images/'
T_graph = np.linspace(0,30,4)*u.degC
G = np.arange(50,5000,50)*u.Hz

def L_Diff(Temperature,G):
  return (((2*u.boltzmann_constant*Temperature * fm.Clay.Diameter*u.m)/(3 * np.pi *pc.viscosity_dynamic_water(Temperature)* (fm.PACl.Diameter*u.m)*G))**(1/3)).to_base_units()

fig, ax = plt.subplots()
for i in range(len(T_graph)):
  ax.semilogx(G,L_Diff(T_graph[i],G).to(u.nm))

ax.legend([*T_graph])
"""* is used to unpack T_graph so that units are preserved when adding another legend item."""


ax.set(xlabel='Velocity gradient (Hz)', ylabel='Diffusion band thickness ($nm$)')
#fig.savefig(imagepath+'Diffusion_band_thickness')
plt.show()

### Example: Nanoparticle attachment time vs Velocity Gradient

The following code can be found in the subsection [Coagulant nanoparticle application](https://aguaclara.github.io/Textbook/Rapid_Mix/RM_Theory_and_Future_Work.html#coagulant-nanoparticle-application)

In [ ]:
import aguaclara.core.physchem as pc
from aguaclara.core.units import unit_registry as u
import aguaclara.research.floc_model as fm
import numpy as np
import matplotlib.pyplot as plt
"""I needed to attach units to material properties due to a bug in floc_model. This will need to be fixed when floc_model is updated."""

#imagepath = 'Rapid_Mix/Images/'

# conventional mechanical design values below
Mix_HRT = np.array([0.5,15,25,35,85])*u.s
Mix_G = np.array([4000,1500,950,850,750])/u.s
Mix_CP = np.multiply(Mix_HRT, np.sqrt(Mix_G))
Mix_Gt = np.multiply(Mix_HRT, Mix_G)
Temperature= 20 * u.degC
Mix_EDR = (Mix_G**2*pc.viscosity_kinematic_water(Temperature))

def Nano_coag_attach_time(pC_CN,C_clay,G,Temperature):
  """We assume that 70% of nanoparticles attach in the average time for one collision."""
  k_nano = 1-np.exp(-1)
  num=2.3*pC_CN*(fm.sep_dist_clay(C_clay,fm.Clay))**2
  den = np.pi * G* k_nano * fm.Clay.Diameter*u.m * L_Diff(Temperature,G)
  return (num/den).to_base_units()

C_Al = 2 * u.mg/u.L
C_clay = 10 * u.NTU
pC_CN = -np.log10(1-0.8)
"""apply 80% of the coagulant nanoparticles to the clay"""

G = np.arange(50,5000,10)*u.Hz

fig, ax = plt.subplots()

for i in range(len(T_graph)):
  ax.semilogx(G,Nano_coag_attach_time(pC_CN,C_clay,G,T_graph[i]))

ax.semilogx(Mix_G.to(1/u.s),Mix_HRT.to(u.s),'o')
ax.legend([*T_graph, "Conventional rapid mix"])
"""* is used to unpack T_graph so that units are preserved when adding another legend item."""


ax.set(xlabel='Velocity gradient (Hz)', ylabel='Nanoparticle attachment time (s)')
#fig.savefig(imagepath+'Coag_attach_time')
plt.show()

### Examples: Energy Tradeoff for Coagulant Transport

The next two examples can be found [here.](https://aguaclara.github.io/Textbook/Rapid_Mix/RM_Theory_and_Future_Work.html#energy-tradeoff-for-coagulant-transport)

In [ ]:
from aguaclara.core.units import unit_registry as u
import aguaclara.research.floc_model as fm
import numpy as np
import aguaclara.core.constants as constants
import aguaclara.core.physchem as pc
import matplotlib.pyplot as plt

Temperature=20* u.degC
Nano_attach_time = Nano_coag_attach_time(pC_CN,C_clay,G,Temperature)

def HL_coag_attach(pC_CN,C_clay,G,Temperature):
  return ((G**2)*pc.viscosity_kinematic_water(Temperature)*Nano_attach_time/(1*u.gravity))

fig, ax = plt.subplots()

for i in range(len(T_graph)):
  ax.loglog(G,HL_coag_attach(pC_CN,C_clay,G,T_graph[i]))

ax.legend(T_graph)

ax.set(xlabel='Velocity gradient (Hz)', ylabel='Head loss (cm)')
#fig.savefig(imagepath+'Coag_attach_head_loss')
plt.show()

In [ ]:
import aguaclara.core.physchem as pc
from aguaclara.core.units import unit_registry as u
import aguaclara.research.floc_model as fm
import numpy as np
import matplotlib.pyplot as plt
"""find G for target head loss"""
HL_nano_transport = np.linspace(10,100,10)*u.cm
def G_max_head_loss(pC_CN,C_clay,HL_nano_transport,Temperature):
  k_nano = 1-np.exp(-1)
  num = u.gravity * HL_nano_transport * np.pi * k_nano
  den= 2.3 * pC_CN * (fm.sep_dist_clay(C_clay,fm.Clay))**2 * pc.viscosity_kinematic_water(Temperature)
  num2 = 2 * u.boltzmann_constant * Temperature
  den2 = 3 * np.pi * pc.viscosity_dynamic_water(Temperature) * (fm.PACl.Diameter*u.m)
  return fm.Clay.Diameter*u.m*((((num/den)**(3) * (num2/den2)).to_base_units())**(1/4))

"""Note the use of to_base_units BEFORE raising to the fractional power. This prevents a rounding error in the unit exponent."""

G_max = G_max_head_loss(pC_CN,C_clay,50*u.cm,Temperature)
print(G_max)

"""The time required?"""
Nano_attach_time = Nano_coag_attach_time(pC_CN,C_clay,G_max,Temperature)
print(Nano_attach_time)
print(G_max*Nano_attach_time)

## Rapid Mix Mechanical Solution

The next code examples are from the [Rapid Mix Design Challenge](https://aguaclara.github.io/Textbook/Rapid_Mix/RM_Mechanical_Solution.html)

In [ ]:
import aguaclara.core.physchem as pc
from aguaclara.core.units import unit_registry as u
import aguaclara.core.utility as ut
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
GraphTempArray =  np.linspace(0,35)*u.degC
GraphKinematicViscosity=pc.viscosity_kinematic_water(GraphTempArray)
fig, ax = plt.subplots()
ax.plot(GraphTempArray, GraphKinematicViscosity.to(u.mm**2/u.s), 'r-')
ax.set(xlabel='Temperature (degrees Celcius)')
ax.set(ylabel='Kinematic viscosity (mm^2/s)')
#fig.savefig('Rapid_Mix/Images/Kinematic_vs_T')
plt.show()

In [ ]:
FlowPlant = 50*(u.L/u.s)
t = 15 * u.s
G = 1500 * 1/(u.s)

def rapid_mix_shaft_power(Flow,VelocityGradient,time,Temp):
    power = ((VelocityGradient**2) * Flow * time * pc.viscosity_kinematic_water(Temp) * pc.density_water(Temp))
    return power


ShaftPower = rapid_mix_shaft_power(FlowPlant,G,t,GraphTempArray)

fig, ax = plt.subplots()
ax.plot(GraphTempArray, ShaftPower.to(u.kW), 'r-')
ax.set(xlabel='Temperature (degrees Celcius)')
ax.set(ylabel='shaft power required (kW)')
#fig.savefig('Rapid_Mix/Images/Shaft_Power')
plt.show()

In [ ]:
TempDesign = u.Quantity(0, u.degC)

RapidMixShaftPower = rapid_mix_shaft_power(FlowPlant,G,t,TempDesign).to(u.hp)
print('The required shaft power is', RapidMixShaftPower)

In [ ]:
ElevDrop = (RapidMixShaftPower/ (FlowPlant * pc.density_water(TempDesign) * u.gravity)).to(u.m)
print('The equivalent height is',  ElevDrop)

In [ ]:
import pandas as pd
#We use a pandas dataframe (df) to hold the data from the excel sheet.
#MotorEfficiencydf = pd.read_excel('motor_efficiency.xlsx')

d = {
    "Horsepower": pd.Series([0.25,0.3333,0.5,0.75,1,1.5,2,3,5,7.5,10,15,25,50,100,200],index=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]),
    "Premium Efficiency": pd.Series([64,68,71,75.5,82.5,82.5,84,86,89.6,90,91.1,91.7,93,94.1,95,95.8],index=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]),

}

MotorEfficiencydf = pd.DataFrame(d)

print(MotorEfficiencydf)

MotorHpArray = np.array(MotorEfficiencydf['Horsepower'])*u.horsepower
MotorEfficiencyArray = np.array(MotorEfficiencydf['Premium Efficiency'])/100

MotorHpArray


In [ ]:
MotorDesign = ut.ceil_nearest(RapidMixShaftPower,MotorHpArray)
print('The rapid mix motor has',  MotorDesign)

In [ ]:
MotorIndex=(np.where(MotorEfficiencydf['Horsepower'] == MotorDesign.magnitude))[0][0]
MotorIndex

In [ ]:
#Examplefor how to use .get_value to return an element by calling on row index and column heading
MotorEfficiency=(MotorEfficiencydf.loc[MotorIndex, 'Premium Efficiency'])/100

#-------------------------------------------Your code below-------------------------------------

print(MotorEfficiencyArray[MotorIndex])

print('The motor efficiency is ',MotorEfficiency,'.')

In [ ]:
COST_MOTOR = 714.64 * u.USD
print('The cost of the motor is', COST_MOTOR)

In [ ]:
from google.colab import files
import pandas as pd
uploaded = files.upload()

import io

ElectricityCostdata = pd.read_csv(io.BytesIO(uploaded['Average_retail_price_of_electricity_monthly.csv']),header=4,nrows=12)

#ElectricityCostdata = pd.read_csv('Average_retail_price_of_electricity_monthly.csv',header=4,nrows=12)
#ElectricityCostdata

In [ ]:
ElectricityRate = np.average(np.array(ElectricityCostdata['commercial cents per kilowatthour']))/100*u.USD/(u.kWh)
print('The New York State commercial electricity cost for the past year was ',ElectricityRate)

In [ ]:
print ('The price of electricity is', ElectricityRate.to(u.USD/u.GJ))

In [ ]:
PowerElectricity = (rapid_mix_shaft_power(FlowPlant,G,t,TempDesign)/MotorEfficiency).to(u.kW)
print ('The power required by the motor is', PowerElectricity)

In [ ]:
rapid_mix_shaft_power(FlowPlant,G,t,TempDesign).to(u.hp)

In [ ]:
RMElectricityCost = (PowerElectricity*ElectricityRate).to(u.USD/u.year)
print ('The cost of electricity to operate the rapid mix unit is', RMElectricityCost)

print(PowerElectricity)
print(ElectricityRate)

In [ ]:
YR25Electricity = RMElectricityCost*(25*u.year)
print ('The cumulative energy costs for a period of 25 years is', YR25Electricity)

## Rapid Mix Design Solution

The following examples are from the subsection [Rapid Mix Design Solution](https://aguaclara.github.io/Textbook/Rapid_Mix/RM_Design_Solution.html#rapid-mix-design-solution)

In [ ]:
import aguaclara.core.physchem as pc
from aguaclara.core.units import unit_registry as u
import aguaclara.core.constants as con
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
DiamJet = 10*u.cm
VelJet = 1*(u.m/u.s)

def energy_dissipation_rate(x, Diam, Velocity):

    dissipation = (50 * Diam**3 * Velocity**3 / ((x - (2 * Diam))**4))
    return dissipation.to(u.m**2/(u.s**3))

DistanceDownstream = (np.array(np.linspace(7,20,100))*DiamJet).to(u.m)

print(energy_dissipation_rate(DistanceDownstream, DiamJet, VelJet)[1])
fig, ax = plt.subplots()
ax.plot(DistanceDownstream, energy_dissipation_rate(DistanceDownstream, DiamJet, VelJet), 'r-')
ax.set(xlabel='Distance from jet origin (m)')
ax.set(ylabel='Energy dissipation rate (W/kg)')
#fig.savefig('Rapid_Mix/Images/Energy_dissipation_rate_vs_Distance_from_jet_origin')
plt.show()